In [3]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #choose whether to use gpu or cpu

In [5]:
data_dir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder'  #path to the folder containing the images
# data_dir = r'C:\Users\Public\PartIIB project 2023_2024\PastData\helical_size'



In [6]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = sorted(os.listdir(root_dir))
        self.labels = [self.extract_label(img) for img in self.images]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images[idx])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label

    def extract_label(self, img_name):
        # Assuming that the label is part of the filename before the first underscore
        label = float(img_name[-17:-5]) #this is the right code, just commented out for testing purposes
        # label = img_name
        return label

# Example usage:
data_transform = transforms.Compose(
[transforms.ToTensor(),
transforms.Normalize((0.5), (0.5))]) 

custom_dataset = CustomImageDataset(root_dir=data_dir, transform=data_transform)

# # Accessing the data
# for img, label in custom_dataset:
#     print(f"Image shape: {img.shape}, Label: {label}")

print(len(custom_dataset))
# train_set, val_set, test_set = random_split(custom_dataset, [int(len(custom_dataset)*0.75), int(len(custom_dataset)*0.15), int(len(custom_dataset)*0.100056)]) #splits data into training, validation and test sets
train_set, test_set = random_split(custom_dataset, [int(len(custom_dataset)*0.75), int(len(custom_dataset)*0.25003)])
print(len(train_set))
# print(len(val_set))
print(len(test_set))

15800
11850
3950


In [7]:
#hyper parameters
num_epochs = 30
batch_size = 1
# learning_rate = 0.0005
learning_rate = 0.001

train = DataLoader(train_set, batch_size=batch_size, shuffle=True)
# val = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test = DataLoader(test_set, batch_size=batch_size, shuffle=False)

class ConvNet(nn.Module): # note need to find out image size
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1,8,10, padding='same') #in_channels, out_channels, kernel_size
        self.normalise1 = nn.BatchNorm2d(8)
        # self.pool = nn.MaxPool2d(5,5) #kernel_size, stride (shift x pixel to the right)
        self.pool1 = nn.AvgPool2d(10, stride=10)
        # self.pool1 = nn.MaxPool2d(10, stride=10)
        self.conv2 = nn.Conv2d(8, 16, 10, padding='same')
        self.normalise2 = nn.BatchNorm2d(16)
        self.pool2 = nn.AvgPool2d(2, stride=2)
        # self.pool2 = nn.MaxPool2d(2, stride=2)
        self.conv3 = nn.Conv2d(16, 32, 10, padding='same')
        self.normalise3 = nn.BatchNorm2d(32) 
        self.conv4 = nn.Conv2d(32, 32, 10, padding='same')
        # self.fc1 = nn.Linear(16*3*3, 120) # 3x3 is the size of the image after 2 conv layers, 16 is the number of channels, 120 is the number of nodes in the hidden layer
        # self.fc2 = nn.Linear(120,84)
        # self.fc3 = nn.Linear(60, 1)
        # self.fc = nn.Linear(32*5*5, 1)
        self.fc1 = nn.Linear(32*5*5, 69)
        self.fc2 = nn.Linear(69,1)
        self.dropout = nn.Dropout(0.2)


    def forward(self, x):
        x = self.pool1(F.relu(self.normalise1(self.conv1(x)))) 
        x = self.pool2(F.relu(self.normalise2(self.conv2(x)))) 
        x = F.relu(self.normalise3(self.conv3(x)))
        x = F.relu(self.normalise3(self.conv4(x)))
        x = x.view(-1, 32*5*5)  #flatten
        x = F.relu(self.dropout(self.fc1(x)))
        x = self.fc2(x)
        # x = self.fc(x)
        
        return x

model = ConvNet().to(device)

# loss and optimizer
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9)
#training loop
n_total_steps = len(train)
for epoch in range(num_epochs):
    total_loss = 0
    for i, (images, labels) in enumerate(train):
        images = images.to(device)
        labels = labels.to(device)

        #forward
        outputs = model(images)
        print(labels)
        labels = labels.float() #this is the right code, just commented out for testing purposes
        # labels = float(labels[0][-17:-5])
        loss = criterion(outputs, labels)

        #backward
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        # if (i+1) % 1000 ==0:
        print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')
        total_loss += loss.item()
    print(f'epoch {epoch+1}/{num_epochs}, average loss = {total_loss/len(train):.4f}')
            

print("Finished Training")

tensor([15.5739, 21.2546, 18.1804, 11.9694,  1.1054,  8.9225, 10.4932, 18.6884,
        11.4817,  2.7948, 22.5144, 10.6223,  6.8011, 12.2798,  2.5879, 12.2444,
        12.0559,  9.2277, 26.1450, 13.1640, 11.3393,  9.2800, 17.9724, 16.0279,
         9.9981, 21.2142,  8.8699, 10.4134,  6.9367,  2.0510,  8.2002,  6.8732],
       dtype=torch.float64)


c:\Users\cs2036\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Convolution.cpp:1009.)
  return F.conv2d(input, weight, bias, self.stride,
c:\Users\cs2036\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 1/30, step 1/371, loss = 172.9656
tensor([12.9058,  1.6271,  2.5361, 11.0478,  9.4349, 14.4903, 13.7149, 17.8673,
        21.0606, 21.1836,  2.9811, 17.0053, 10.5590, 10.2741,  4.2664,  2.1660,
        17.3053,  6.8481,  9.4089, 22.1634,  7.0666, 10.2943,  5.7995,  0.9937,
        13.5532,  7.0493, 20.2966,  6.9667, 31.9257, 16.8129,  8.7910, 14.3228],
       dtype=torch.float64)
epoch 1/30, step 2/371, loss = 116.8230
tensor([11.3597, 22.4637, 15.1900,  6.3348, 16.4906,  2.9760,  2.2638, 21.1777,
        13.8555, 11.8144,  5.9594, 10.7591,  9.4086, 11.6918, 16.9340,  8.9416,
         3.8501, 11.4184, 15.1298, 16.2811, 10.0587,  7.9765,  9.9173,  7.4251,
        13.9203,  3.0499,  6.8843,  9.7388,  6.4179, 16.6822, 16.4699,  7.9612],
       dtype=torch.float64)
epoch 1/30, step 3/371, loss = 52.3215
tensor([14.7597,  9.4696,  9.7953,  1.0385,  7.0581, 27.6137, 11.9389,  8.0568,
        11.7501, 14.6828, 33.7881, 10.4658, 19.6461,  7.5627,  1.4297, 12.8357,
        19.0598,  9.413

c:\Users\cs2036\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 2/30, step 1/371, loss = 42.2280
tensor([ 1.3413,  5.6888, 22.3018, 12.5426,  8.2470, 11.2792, 13.1323, 16.1432,
        16.5882, 11.4303, 30.9749, 11.3433, 16.9089, 19.0343,  7.7405, 19.6765,
        29.5016, 13.9284, 12.1304, 16.1480,  4.6148, 15.9136, 21.0039, 12.7452,
         6.6302, 17.7245, 13.1027, 17.6169,  8.7480, 13.1588,  9.7420, 16.3795],
       dtype=torch.float64)
epoch 2/30, step 2/371, loss = 44.7227
tensor([10.3234, 19.4746,  3.3407,  9.4908, 10.7197, 11.6875, 16.6688,  7.1658,
         7.9537,  7.4982, 20.7438, 12.9045, 24.2538, 28.5837, 15.0946,  9.0090,
        10.8949, 17.6096, 10.9484, 11.8335,  8.9552,  9.4625, 18.2138, 11.8176,
         9.6612,  7.7373,  0.8691,  1.0951,  2.8645, 26.9138,  7.8160, 13.3692],
       dtype=torch.float64)
epoch 2/30, step 3/371, loss = 44.5152
tensor([13.1510, 25.6404,  6.2177,  8.9243, 16.6840, 13.3617,  7.7843,  2.7766,
         0.9520, 10.7576, 33.0243, 16.7368, 14.2640,  1.0214, 20.7162, 18.0709,
         2.7776,  1.8911,

In [1]:
with torch.no_grad(): # no need to calculate gradient
    squared_difference = 0
    for images, labels in test:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        #value, index
        _, predictions = torch.max(outputs, 1)
        squared_difference += (predictions - labels) ** 2
    
    rmse = torch.sqrt(squared_difference / len(test))
    print(f'RMSE = {rmse}')

NameError: name 'torch' is not defined

In [ ]:
import time
from tqdm import tqdm

count =0
for i in tqdm(range(100)):
    count += 1
    time.sleep(1)
print(count)